In [8]:
from pathlib import Path
import pickle
import ipdb
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
dir_path = Path("/home/chenhaojun/DIffUCO/Checkpoints/z5xmvb6v/visualization/0")

jgraph_path = dir_path / "jgraph.pkl"
solution_path = dir_path / "solution.pkl"

with open(jgraph_path, 'rb') as f:
    jgraph = pickle.load(f)

with open(solution_path, 'rb') as f:
    solution = pickle.load(f)


G = nx.DiGraph()
for i, nodes in enumerate(jgraph.nodes):
    G.add_node(i)
    
for send, receive in zip(jgraph.senders, jgraph.receivers):
    G.add_edge(send, receive)

independent_set = solution.flatten()[:len(jgraph.nodes)]
node_colors = ['red' if independent_set[i] == 1 else 'black' for i in range(len(independent_set))]


pos = nx.spring_layout(G) 
nx.draw(G, pos, node_color=node_colors, with_labels=True, node_size=len(node_colors), font_color='white')

# # 保存图到文件
# plt.savefig("MIS.png")
# plt.close()

FileNotFoundError: [Errno 2] No such file or directory: '/home/chenhaojun/DIffUCO/Checkpoints/z5xmvb6v/visualization/0/jgraph.pkl'

In [9]:
def is_independent_set(jgraph, independent_set):
    is_MIS = True
    for send, receive in zip(jgraph.senders, jgraph.receivers):
        if independent_set[send] == 1 and independent_set[receive] == 1:
            is_MIS = False
    # if is_MIS:
    #     print(f"The size is {sum(independent_set)}")
    return is_MIS

In [10]:
def count_voilate(jgraph, independent_set):
    is_MIS = True
    edges = []
    voilate_cnt = 0
    for send, receive in zip(jgraph.senders, jgraph.receivers):
        if (send, receive) in edges or (receive, send) in edges:
            continue
        edges.append((send, receive))
        if independent_set[send] == 1 and independent_set[receive] == 1:
            voilate_cnt += 1
    return voilate_cnt

In [5]:
# def calc_IS_size():
#     path = ""
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/z5xmvb6v/visualization")
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/xvrrfwsg/visualization")
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/xvrrfwsg/visualization_KS_3_1000")
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/spcrakpu/visualization_KS_3_1000")
# path = Path("/home/haojun/DIffUCO/Checkpoints/71qtteyy/visualization_KS_3_1000")
path = Path("/home/haojun/DIffUCO/Checkpoints/71qtteyy/visualization_KS_3_1000_train")
cnt = []
tot_nodes = []
for graph_dir in tqdm(path.iterdir()):
    jgraph_path = graph_dir / "jgraph.pkl"
    solution_path = graph_dir / "solution.pkl"

    with open(jgraph_path, 'rb') as f:
        jgraph = pickle.load(f)

    with open(solution_path, 'rb') as f:
        solution = pickle.load(f)

    G = nx.DiGraph()
    for i, nodes in enumerate(jgraph.nodes):
        G.add_node(i)
        
    for send, receive in zip(jgraph.senders, jgraph.receivers):
        G.add_edge(send, receive)
    
    print(f"solution 's size={len(solution.flatten())}")

    independent_set = solution.flatten()[:len(jgraph.nodes)]
    if is_independent_set(jgraph, independent_set):
        cnt.append(sum(independent_set))
        tot_nodes.append(len(independent_set))
    # cnt.append(sum(independent_set))
    # tot_nodes.append(len(independent_set))
cnt = np.array([np.array(ele) for ele in cnt])
print(f"cnt={cnt}")
print(f"tot_nodes={tot_nodes}")

0it [00:00, ?it/s]2024-07-12 11:53:38.220173: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


solution 's size=1021


0it [00:10, ?it/s]


KeyboardInterrupt: 

In [11]:
# def calc_IS_size():
#     path = ""
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/z5xmvb6v/visualization")
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/xvrrfwsg/visualization")
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/xvrrfwsg/visualization_KS_3_1000")
# path = Path("/home/chenhaojun/DIffUCO/Checkpoints/spcrakpu/visualization_KS_3_1000")
path = Path("/home/haojun/DIffUCO/Checkpoints/71qtteyy/visualization_KS_3_1000_train")
cnt = []
tot_nodes = []
for graph_dir in tqdm(path.iterdir()):
    jgraph_path = graph_dir / "jgraph.pkl"
    solution_path = graph_dir / "solution.pkl"

    with open(jgraph_path, 'rb') as f:
        jgraph = pickle.load(f)

    with open(solution_path, 'rb') as f:
        solution = pickle.load(f)

    G = nx.DiGraph()
    for i, nodes in enumerate(jgraph.nodes):
        G.add_node(i)
        
    for send, receive in zip(jgraph.senders, jgraph.receivers):
        G.add_edge(send, receive)
    
    print(f"solution 's size={len(solution.flatten())}")

    independent_set = solution.flatten()[:len(jgraph.nodes)]

    cnt.append(sum(independent_set))
    tot_nodes.append(len(independent_set))
    print(f"the number of violate edges = {count_voilate(jgraph, independent_set)}")
    print(f"size of independent set = {sum(independent_set)}")
    print(f"size of nodes = {len(independent_set)}")
    

0it [00:00, ?it/s]

solution 's size=1021


1it [05:49, 349.21s/it]

the number of violate edges = 16
size of independent set = 13
size of nodes = 1000
solution 's size=1021


1it [10:03, 603.02s/it]


KeyboardInterrupt: 

In [13]:
for send, receive in zip(jgraph.senders, jgraph.receivers):
    print(send, receive)

0 1
0 17
0 22
0 27
0 28
0 36
0 39
0 42
0 44
0 49
0 50
0 53
0 55
0 62
0 64
0 66
0 68
0 80
0 81
0 82
0 84
0 86
0 87
0 88
0 91
0 96
0 97
0 99
0 107
0 110
0 116
0 123
0 131
0 133
0 138
0 142
0 145
0 149
0 154
0 156
0 166
0 167
0 169
0 172
0 173
0 180
0 181
0 184
0 187
0 192
0 198
0 211
0 212
0 221
0 223
0 227
0 234
0 236
0 237
0 238
0 243
0 247
0 249
0 250
0 251
0 252
0 256
0 257
0 259
0 262
0 263
0 265
0 266
0 268
0 274
0 276
0 278
0 283
0 289
0 291
0 293
0 295
0 301
0 302
0 309
0 311
0 312
0 313
0 317
0 320
0 321
0 322
0 323
0 326
0 333
0 339
0 343
0 345
0 347
0 357
0 362
0 367
0 371
0 372
0 380
0 384
0 396
0 420
0 421
0 426
0 427
0 442
0 454
0 456
0 462
0 469
0 478
0 480
0 482
0 486
0 487
0 491
0 496
0 498
0 499
0 500
0 507
0 512
0 513
0 516
0 523
0 527
0 528
0 530
0 537
0 538
0 540
0 546
0 553
0 558
0 562
0 567
0 575
0 576
0 577
0 578
0 579
0 581
0 583
0 585
0 587
0 589
0 592
0 601
0 606
0 607
0 608
0 609
0 615
0 618
0 621
0 622
0 623
0 624
0 630
0 632
0 635
0 638
0 650
0 663
0 664
0 6

In [14]:
sum(independent_set)

Array(15, dtype=int32)

In [15]:
len(independent_set)

1000

In [17]:
edges = []
voilate_cnt = 0
for send, receive in zip(jgraph.senders, jgraph.receivers):
    if (send, receive) in edges or (receive, send) in edges:
        continue
    edges.append((send, receive))
    if independent_set[send] == 1 and independent_set[receive] == 1:
        voilate_cnt += 1
        print(f"send={send},receive={receive}")

send=49,receive=80
send=49,receive=567
send=80,receive=550
send=80,receive=567
send=196,receive=813
send=425,receive=481
send=425,receive=887
send=425,receive=923
send=481,receive=887
send=481,receive=923
send=550,receive=591
send=606,receive=801
send=887,receive=923
